In [1]:
import pandas as pd
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from googletrans import Translator, constants
from nltk.tokenize.treebank import TreebankWordDetokenizer
from tqdm.notebook import tqdm
import string

In [2]:
data = pd.read_csv("data/tweets_bf_stpwrd.csv")
data.head(10)
# data.drop(["sentimen"], axis = 1, inplace = True)

,tweet
0,"['pembelajaran', 'tatap', 'muka', 'terbatas', ..."
1,"['pembelajaran', 'tatap', 'muka', 'terbatas', ..."
2,"['satgas', 'covid', 'dukung', 'perguruan', 'ti..."
3,"['lagi', 'dan', 'lagi', 'pembelajaran', 'tatap..."
4,"['anakanak', 'harus', 'dipastikan', 'siap', 'm..."
5,"['pembelajaran', 'tatap', 'muka', 'ptm', 'perl..."
6,"['pembelajaran', 'tatap', 'muka', 'siap', 'dil..."
7,"['ingin', 'menggelar', 'pembelajaran', 'tatap'..."
8,"['bupati', 'sleman', 'dra', 'hj', 'kustini', '..."
9,"['pembelajaran', 'tatap', 'muka', 'ptm', 'perl..."


In [3]:
temp_detokenize = []

def detokenize(text):
    text1 = text.replace(']','').replace('[','')
    arr = text1.replace('"','').replace("\'","").split(",")
    return(TreebankWordDetokenizer().detokenize(arr))

data['tweet'] = data['tweet'].astype('U').apply(detokenize)

In [4]:
temp_translate = []
translator = Translator()
data_len = len(data)

for tweet in tqdm(data['tweet'],desc='Total progress'):
    all_polarity = 0
    translation = translator.translate(tweet)
    translated_word = translation.text
    anal = TextBlob(translated_word, analyzer=NaiveBayesAnalyzer())
    all_polarity += anal.polarity
    if (all_polarity/100 > 0):
        an_translate = "Positif"
    elif (all_polarity/100 == 0):
        an_translate = "Netral"
    else:
        an_translate = "Negatif"
    temp_translate.append([tweet, an_translate, all_polarity])

data_anal = pd.DataFrame(temp_translate, columns=['tweet', 'sentimen','polaritas'])

Total progress:   0%|          | 0/1001 [00:00<?, ?it/s]

In [5]:
data_anal.head()

,tweet,sentimen,polaritas
0,pembelajaran tatap muka terbatas wajib te...,Negatif,-0.071429
1,pembelajaran tatap muka terbatas wajib te...,Negatif,-0.071429
2,satgas covid dukung perguruan tinggi mela...,Netral,0.000000
3,lagi dan lagi pembelajaran tatap muka pt...,Positif,0.071429
4,anakanak harus dipastikan siap mengikuti ...,Positif,0.200000
...,...,...,...
295,menteri pendidikan kebudayaan riset dan t...,Netral,0.000000
296,karina ptm pembelajaran tatap muka,Netral,0.000000
297,pembelajaran tatap muka perlu dilakukan m...,Netral,0.000000
298,pembelajaran tatap muka dapat mendorong m...,Netral,0.000000


In [6]:
data_anal.to_csv('test_textblob.csv',index=False)